# **Finetune Piper Vietnamese and export to ONNX **


# **Install packages**

In [ ]:
# Setup Evironments
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime==1.15.1 pytorch-lightning==1.7.0 torch==1.11.0
!pip install -q torchtext==0.12.0 torchvision==0.12.0
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!pip install -q textgrid
!bash build_monotonic_align.sh
!apt-get install -q espeak-ng

In [ ]:
# This is serious bug, update pip with 24.0.0 than pytorch-lightning can be installed !!!
!pip install pip==24.0.0
!pip install pytorch-lightning==1.7.0 onnxruntime piper_phonemize

# **Download datasets**

In [ ]:
!gdown 1-MqCJKtZojWmCESVEYVxMEzqa8oc3YfO
!unzip thanhdata1111.zip -d wav/

# **Data preprocessing**

# **Resampling datasets**

In [ ]:
# Convert to sample rate 22050 for high quality
resample = True
sample_rate = 22050
if resample:
  !python resample.py --input_dir "wav" --output_dir "wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv wavs_resampled wav

In [ ]:
!cp wav/metadata.csv /content/piper/src/python

# **Preprocess text to Phonemes**

In [ ]:
# Convert text to phonemize format
dataset_format = "ljspeech"
force_sp = "--single-speaker"
!python -m piper_train.preprocess \
  --language vi \
  --input-dir /content/piper/src/python \
  --output-dir /content/piper/src/python \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

# **Download pretrained model**

In [ ]:
# Download pretrain model from piper
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/vi/vi_VN/vais1000/medium/epoch%3D4769-step%3D919580.ckpt
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/raw/main/vi/vi_VN/vais1000/medium/config.json

In [ ]:
# Rename pretrain model for easy
!mv epoch=4769-step=919580.ckpt pretrained_vi.ckpt

In [ ]:
!pip uninstall torchmetrics
!pip install torchmetrics==0.11.4

# **Training model**

Setup params and ready for training

In [ ]:
!python -m piper_train \
    --dataset-dir "/content/piper/src/python/" \
    --accelerator "gpu" \
    --devices 1 \
    --batch-size 30 \
    --validation-split 0.0 \
    --num-test-examples 2 \
    --quality "medium" \
    --checkpoint-epochs 5 \
    --log_every_n_steps 1 \
    --max_epochs 4799 \
    --resume_from_checkpoint "/content/piper/src/python/pretrained_vi.ckpt" \
    --precision 32


# **Export ONNX model**

In [ ]:
# Export finetuned model to onnx
!python3 -m piper_train.export_onnx \
    /content/piper/src/python/lightning_logs/version_0/checkpoints/epoch=4794-step=919680.ckpt \
    /content/finetuning_pretrained_vi.onnx

!cp /content/piper/src/python/config.json \
   /content/finetuning_pretrained_vi.onnx.json

In [ ]:
# Export pretrained model to onnx
    #/content/piper/src/python/lightning_logs/version_0/checkpoints/epoch=4799-step=919700.ckpt \
!python3 -m piper_train.export_onnx \
    /content/piper/src/python/pretrained_vi.ckpt \
    /content/pretrained_vi.onnx

!cp /content/piper/src/python/config.json \
   /content/pretrained_vi.onnx.json

# **Test inference**

In [ ]:
!pip install piper-tts

In [ ]:
!echo 'Trong tổng số khoảng 15 triệu cử tri người Mỹ gốc Á trong kỳ bầu cử tổng thống Mỹ năm 2024, số cử tri gốc Việt chỉ hơn 1 triệu người. Ở những kỳ bầu cử gần nhất, cử tri gốc Việt bỏ phiếu ủng hộ cho ứng viên nào, họ quan tâm đến những vấn đề gì và năm nay, họ có xu hướng ủng hộ ai nhiều hơn? Nghiên cứu sinh tiến sĩ Nguyễn Ngọc Yến My (Chuyên ngành Truyền thông và Công vụ - ĐH bang Louisiana, Mỹ) dẫn một số nguồn nghiên cứu và khảo sát để trả lời cho các câu hỏi này.' | \
  piper -m /content/finetuning_pretrained_vi.onnx --output_file /content/test1.wav

In [ ]:
!echo 'Dự án đường sắt tốc độ cao Bắc Nam dự kiến dài 1.541 km, qua 20 tỉnh thành với tốc độ thiết kế 350 km/h, đi từ Hà Nội đến TP HCM hết 5,5 giờ, tiết kiệm thời gian 6 lần so với tàu hỏa thường.' | \
  piper -m /content/finetuning_pretrained_vi.onnx --output_file /content/test2.wav

In [ ]:
# Quality of finetuned model
from IPython.display import Audio
Audio("/content/test2.wav")

In [ ]:
from google.colab import files
# Trigger the download
files.download('/content/test2.wav')

In [ ]:
!echo 'Dự án đường sắt tốc độ cao Bắc Nam dự kiến dài 1.541 km, qua 20 tỉnh thành với tốc độ thiết kế 350 km/h, đi từ Hà Nội đến TP HCM hết 5,5 giờ, tiết kiệm thời gian 6 lần so với tàu hỏa thường.' | \
  piper -m /content/pretrained_vi.onnx --output_file /content/test_original.wav

In [ ]:
from google.colab import drive
from google.colab import files
import shutil

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Define the path in Google Drive where you want to save the file
drive_path = '/content/drive/MyDrive/'

# Step 3: Move the uploaded file to your Google Drive folder
shutil.move('/content/piper/src/python/lightning_logs/version_0/checkpoints/epoch=4794-step=919680.ckpt', drive_path)

print(f"File successfully uploaded to: {drive_path}")
